方案根据官方提供的baseline改进

1. 数据路径处理，tool/dataset.py修改，训练时需要修改第43行增加ann_file = ann_file.replace('traffic5','')，第106行增加 path = path.replace('traffic5','')
2. 如果需要预测b榜数据,上述tool/dataset.py文件取消训练时增加的43行及106行，并去掉第42行及第105行的注释,需修改test_final.py第96行测试集数量584改为10066
3. 模型为resnext101,需要双卡P100
4. 修改faster_rcnn_resx101_800size_trafficdet_demo参数，主要为增大尺寸

In [1]:
!ls /home/megstudio/dataset/dataset-2805

annotations  images


In [2]:
!cd ~/workspace; git clone https://github.com/er-muyue/megengine-trafficsign.git

Cloning into 'megengine-trafficsign'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 65 (delta 25), reused 58 (delta 21), pack-reused 0
Receiving objects: 100% (65/65), 46.43 KiB | 399.00 KiB/s, done.
Resolving deltas: 100% (25/25), done.


In [3]:
!mkdir ~/workspace/submit

In [4]:
!wget https://data.megengine.org.cn/models/weights/faster_rcnn_res50_coco_3x_800size_40dot1_8682ff1a.pkl; mkdir ~/workspace/megengine-trafficsign/trafficdet/weights; mv faster_rcnn_res50_coco_3x_800size_40dot1_8682ff1a.pkl ~/workspace/megengine-trafficsign/trafficdet/weights/;

--2021-09-23 00:54:59--  https://data.megengine.org.cn/models/weights/faster_rcnn_res50_coco_3x_800size_40dot1_8682ff1a.pkl
Resolving data.megengine.org.cn (data.megengine.org.cn)... 106.117.214.222, 106.117.214.217, 106.117.214.219, ...
Connecting to data.megengine.org.cn (data.megengine.org.cn)|106.117.214.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167248318 (160M) [application/octet-stream]
Saving to: ‘faster_rcnn_res50_coco_3x_800size_40dot1_8682ff1a.pkl’

faster_rcnn_res50_c 100%[===================>] 159.50M  12.8MB/s    in 12s     

2021-09-23 00:55:12 (13.8 MB/s) - ‘faster_rcnn_res50_coco_3x_800size_40dot1_8682ff1a.pkl’ saved [167248318/167248318]



In [ ]:
!ls ~/workspace/megengine-trafficsign/trafficdet/weights

In [5]:
!wget https://data.megengine.org.cn/models/weights/faster_rcnn_resx101_coco_2x_800size_44dot1_e5e0060b.pkl; mv faster_rcnn_resx101_coco_2x_800size_44dot1_e5e0060b.pkl ~/workspace/megengine-trafficsign/trafficdet/weights/;

--2021-09-23 01:13:49--  https://data.megengine.org.cn/models/weights/faster_rcnn_resx101_coco_2x_800size_44dot1_e5e0060b.pkl
Resolving data.megengine.org.cn (data.megengine.org.cn)... 124.238.244.238, 124.238.244.237, 124.238.244.236, ...
Connecting to data.megengine.org.cn (data.megengine.org.cn)|124.238.244.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 420805451 (401M) [application/octet-stream]
Saving to: ‘faster_rcnn_resx101_coco_2x_800size_44dot1_e5e0060b.pkl’

faster_rcnn_resx101 100%[===================>] 401.31M  13.0MB/s    in 30s     

2021-09-23 01:14:19 (13.3 MB/s) - ‘faster_rcnn_resx101_coco_2x_800size_44dot1_e5e0060b.pkl’ saved [420805451/420805451]



In [4]:
!export PYTHONPATH=~/workspace/megengine-trafficsign/trafficdet:$PYTHONPATH; cd ~/workspace/megengine-trafficsign/trafficdet; python3 tools/train.py -n 2 -b 1 -f configs/faster_rcnn_resx101_800size_trafficdet_demo.py -d ~/dataset/dataset-2805 -w weights/faster_rcnn_resx101_coco_2x_800size_44dot1_e5e0060b.pkl

23 01:30:07 Device Count = 2
23 01:30:10 load_serialized_obj_from_url: download to or using cached /home/megstudio/.cache/megengine/serialized/d3d065_resnext101_32x8d_fbaug_79520_80efb344.pkl
23 01:30:10 load_serialized_obj_from_url: download to or using cached /home/megstudio/.cache/megengine/serialized/d3d065_resnext101_32x8d_fbaug_79520_80efb344.pkl
23 01:30:14 Loading Base-Pretrain weights...
23 01:30:14 ------------------------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------
backbone                  resnext101_32x8d
backbone_pretrained       True
backbone_norm             FrozenBN
backbone_freeze_at        2
fpn_norm                  <class 'NoneType'>
fpn_in_features           ['res2', 'res3', 'res4', 'res5']
fpn_in_strides            [4, 8, 16, 32]
fpn_in_channels           [256, 512, 1024, 2048]
fpn_out_channels          256
train_dataset             {'name':

In [6]:
!export PYTHONPATH=~/workspace/megengine-trafficsign/trafficdet:$PYTHONPATH; cd ~/workspace/megengine-trafficsign/trafficdet; python3 tools/test.py -n 2 -se 12 -f configs/faster_rcnn_resx101_800size_trafficdet_demo.py -d ~/dataset/dataset-2805

100%|█████████████████████████████████████████| 299/299 [02:46<00:00,  1.80it/s]
23 08:13:55 Save to logs/faster_rcnn_resx101_800size_trafficdet_demo_gpus2/epoch_12.json finished, start evaluation!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.11s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.95s).
Accumulating evaluation results...
DONE (t=0.26s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.565
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.811
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.627
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.453
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.649
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.730
 Average Recall     (AR) @[

In [7]:
!export PYTHONPATH=~/workspace/megengine-trafficsign/trafficdet:$PYTHONPATH; cd ~/workspace/megengine-trafficsign/trafficdet; python3 tools/test.py -n 2 -se 13 -f configs/faster_rcnn_resx101_800size_trafficdet_demo.py -d ~/dataset/dataset-2805

100%|█████████████████████████████████████████| 299/299 [01:34<00:00,  3.16it/s]
23 07:17:39 Save to logs/faster_rcnn_resx101_800size_trafficdet_demo_gpus2/epoch_13.json finished, start evaluation!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.12s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.80s).
Accumulating evaluation results...
DONE (t=0.23s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.566
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.802
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.635
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.445
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.650
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.740
 Average Recall     (AR) @[

In [7]:
!export PYTHONPATH=~/workspace/megengine-trafficsign/trafficdet:$PYTHONPATH; cd ~/workspace/megengine-trafficsign/trafficdet; python3 tools/test.py -n 2 -se 14 -f configs/faster_rcnn_resx101_800size_trafficdet_demo.py -d ~/dataset/dataset-2805

100%|█████████████████████████████████████████| 299/299 [02:33<00:00,  1.95it/s]
23 08:17:17 Save to logs/faster_rcnn_resx101_800size_trafficdet_demo_gpus2/epoch_14.json finished, start evaluation!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.11s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.78s).
Accumulating evaluation results...
DONE (t=0.22s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.570
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.811
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.650
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.457
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.649
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.738
 Average Recall     (AR) @[

In [9]:
!export PYTHONPATH=~/workspace/megengine-trafficsign/trafficdet:$PYTHONPATH; cd ~/workspace/megengine-trafficsign/trafficdet; python3 tools/test.py -n 2 -se 15 -f configs/faster_rcnn_resx101_800size_trafficdet_demo.py -d ~/dataset/dataset-2805

100%|█████████████████████████████████████████| 299/299 [01:34<00:00,  3.15it/s]
23 07:21:05 Save to logs/faster_rcnn_resx101_800size_trafficdet_demo_gpus2/epoch_15.json finished, start evaluation!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.09s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.64s).
Accumulating evaluation results...
DONE (t=0.23s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.561
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.794
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.640
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.439
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.647
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.728
 Average Recall     (AR) @[

In [12]:
!export PYTHONPATH=~/workspace/megengine-trafficsign/trafficdet:$PYTHONPATH; cd ~/workspace/megengine-trafficsign/trafficdet; python3 tools/test.py -n 2 -se 14 -f configs/faster_rcnn_resx101_800size_trafficdet_demo.py -d ~/dataset/dataset-2805

100%|█████████████████████████████████████████| 299/299 [02:24<00:00,  2.07it/s]
23 08:32:58 Save to logs/faster_rcnn_resx101_800size_trafficdet_demo_gpus2/epoch_14.json finished, start evaluation!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.15s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.41s).
Accumulating evaluation results...
DONE (t=0.37s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.567
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.816
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.642
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.446
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.651
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.743
 Average Recall     (AR) @[

In [ ]:
!export PYTHONPATH=~/workspace/megengine-trafficsign/trafficdet:$PYTHONPATH; cd ~/workspace/megengine-trafficsign/trafficdet; python3 tools/test_final.py -n 2 -se 14 -f configs/faster_rcnn_resx101_800size_trafficdet_demo.py -d ~/dataset/traffic5/

100%|███████████████████████████████████| 10066/10066 [1:20:28<00:00,  2.08it/s]


In [9]:
!cp -r ~/workspace/megengine-trafficsign/trafficdet/logs/faster_rcnn_resx101_800size_trafficdet_demo_gpus2/test_final_epoch_14.json ~/workspace/submit/results.json

In [8]:
1

1